# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods 
Now that you have been equipped with the skills and the tools to use location data to explore a geographical location, over the course of two weeks, you will have the opportunity to be as creative as you want and come up with an idea to leverage the Foursquare location data to explore or compare neighborhoods or cities of your choice or to come up with a problem that you can use the Foursquare location data to solve.

### 1) Introduction/Business Problem

The idea of this study is to help people planning to open a new restaurant in Toronto to chose the right location by providing data about the income and population of each neighborhood as well as the competitors already present on the same regions. 
The challenge to resolve is being able to find a location to open new restaurant in Toranto that offers similar characteristics and benefits to my current situation.

Therefore, in order to set a basis for comparison, I want to open vietnamese restaurant and base on the following conditions :
1. With population over 20000
2. High Income
3. There are not or few Vietnamese restaurants 



### 2) Data
Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

To provide the stakeholders the necessary information I'll be combining Toronto's 2016 Census that contains Population, Average income per Neighborhood with Foursquare API to collect competitors on the same neighborhoods.

Toronto's Census data in publicly available at this website: https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8c732154-5012-9afe-d0cd-ba3ffc813d5a



Before we get the data and start exploring it, let's download all the dependencies that we will need.

### 3.Methodology section

The strategy is based on mapping the above described data in section 2.0, in order to facilitate the choice of at least two candidate places for rent new restaurant.
The procesing of these DATA and its mapping will allow to answer the key questions to make a decision:
1. How much income of place that open new restaurant? 
2. What is density restauran of each place?
3. How much population of each place?
4. What country cuisine have most?

Then I will choose 5 candidate is best base on business introduction. 

In [1]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


### 2.1) Download and Explore Dataset
Using data from 2016 census to collect Toronto's income per neighborhood

In [2]:
# Toronto Open Data Catalogue - Neighbourhood Profiles 2016 (CSV)
# https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8c732154-5012-9afe-d0cd-ba3ffc813d5a

csv_path='https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df = pd.read_csv(csv_path,encoding='latin1')
print('Data loaded')

Data loaded


In [3]:
Neighbourhoods = list(df.columns.values)
Neighbourhoods = Neighbourhoods[5:]
dfToronto = pd.DataFrame(index = Neighbourhoods,columns=["code_id","Population_2016","Income_2016"])
# Population_2016 = Population, 2016
# Income_2016 = Total income: Average amount ($)

for index, row in dfToronto.iterrows():
    dfToronto.at[index, 'code_id'] = df[index][0]
    dfToronto.at[index, 'Population_2016'] = df[index][2]
    dfToronto.at[index, 'Income_2016'] = df[index][2264]

In [4]:
dfToronto["Population_2016"] = list(map(lambda x : x.replace(",",""),dfToronto["Population_2016"]))
dfToronto["Income_2016"] = list(map(lambda x : x.replace(",",""),dfToronto["Income_2016"]))

In [5]:
dfToronto = dfToronto.astype(int)
dfToronto = dfToronto.sort_values(by = "code_id")
dfToronto = dfToronto.reset_index().rename(columns = {"index" : "Neighbourhoods"})
dfToronto.head()

,Neighbourhoods,code_id,Population_2016,Income_2016
0,West Humber-Clairville,1,33312,31771
1,Mount Olive-Silverstone-Jamestown,2,32954,26548
2,Thistletown-Beaumond Heights,3,10360,32815
3,Rexdale-Kipling,4,10529,34418
4,Elms-Old Rexdale,5,9456,32012


In [6]:
# load  long,lat data, link : https://open.toronto.ca/dataset/neighbourhoods/
df_geo = pd.read_csv("Neighbourhoods.csv")
df_geo.head(5)

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,2801,25886861,25926662,49885,94,94,Wychwood (94),Wychwood (94),NaN,NaN,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"{u'type': u'Polygon', u'coordinates': (((-79.4..."
1,2802,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),NaN,NaN,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"{u'type': u'Polygon', u'coordinates': (((-79.4..."
2,2803,25886834,25926664,49885,97,97,Yonge-St.Clair (97),Yonge-St.Clair (97),NaN,NaN,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"{u'type': u'Polygon', u'coordinates': (((-79.3..."
3,2804,25886593,25926665,49885,27,27,York University Heights (27),York University Heights (27),NaN,NaN,-79.488883,43.765736,16491553,2.541821e+07,25632.335242,"{u'type': u'Polygon', u'coordinates': (((-79.5..."
4,2805,25886688,25926666,49885,31,31,Yorkdale-Glen Park (31),Yorkdale-Glen Park (31),NaN,NaN,-79.457108,43.714672,16491569,1.156669e+07,13953.408098,"{u'type': u'Polygon', u'coordinates': (((-79.4..."


In [7]:
df_geo_toronto = pd.DataFrame(columns=["code_id","Longitude","Latitude"])
df_geo_toronto["code_id"] = df_geo["AREA_SHORT_CODE"]
df_geo_toronto["Longitude"] = df_geo["LONGITUDE"]
df_geo_toronto["Latitude"] = df_geo["LATITUDE"]

In [90]:
df_geo_toronto = df_geo_toronto.sort_values(by = "code_id")
df_geo_toronto.head()

,code_id,Longitude,Latitude
63,1,-79.596356,43.716180
20,2,-79.587259,43.746868
56,3,-79.563491,43.737988
40,4,-79.566228,43.723725
112,5,-79.548983,43.721519


### merge data 

In [8]:
df_new = pd.merge(dfToronto, df_geo_toronto[["code_id","Longitude","Latitude"]], on = "code_id")

In [9]:
df_new.to_csv("Toronto.csv")
df_new.head()

,Neighbourhoods,code_id,Population_2016,Income_2016,Longitude,Latitude
0,West Humber-Clairville,1,33312,31771,-79.596356,43.716180
1,Mount Olive-Silverstone-Jamestown,2,32954,26548,-79.587259,43.746868
2,Thistletown-Beaumond Heights,3,10360,32815,-79.563491,43.737988
3,Rexdale-Kipling,4,10529,34418,-79.566228,43.723725
4,Elms-Old Rexdale,5,9456,32012,-79.548983,43.721519


In [10]:
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [11]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [12]:
map_Toranto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Neighbourhoods']):
    label = "{}".format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toranto)  
    
map_Toranto

In [13]:
CLIENT_ID = '00UX0ISEZI0YPT5NF3HENXKV50IIO1E4OHS4WK4Y0EEY5QZA' # your Foursquare ID
CLIENT_SECRET = 'QYHPJNHQWWSIQ5LNXX5GUYB1L4LIUQTV3BHXHJESLOUU2HKF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 
radius = 500
latitude = df_new.loc[0,'Latitude'] 
longitude = df_new.loc[0,'Longitude']
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 00UX0ISEZI0YPT5NF3HENXKV50IIO1E4OHS4WK4Y0EEY5QZA
CLIENT_SECRET:QYHPJNHQWWSIQ5LNXX5GUYB1L4LIUQTV3BHXHJESLOUU2HKF


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhoods', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
import requests # Library for web scraping
names = df_new['Neighbourhoods']
latitudes = df_new['Latitude']
longitudes = df_new['Longitude']
toronto_venues = getNearbyVenues(names,latitudes,longitudes)

West Humber-Clairville
Mount Olive-Silverstone-Jamestown
Thistletown-Beaumond Heights
Rexdale-Kipling
Elms-Old Rexdale
Kingsview Village-The Westway
Willowridge-Martingrove-Richview
Humber Heights-Westmount
Edenbridge-Humber Valley
Princess-Rosethorn
Eringate-Centennial-West Deane
Markland Wood
Etobicoke West Mall
Islington-City Centre West
Kingsway South
Stonegate-Queensway
Mimico (includes Humber Bay Shores)
New Toronto
Long Branch
Alderwood
Humber Summit
Humbermede
Pelmo Park-Humberlea
Black Creek
Glenfield-Jane Heights
Downsview-Roding-CFB
York University Heights
Rustic
Maple Leaf
Brookhaven-Amesbury
Yorkdale-Glen Park
Englemount-Lawrence
Clanton Park
Bathurst Manor
Westminster-Branson
Newtonbrook West
Willowdale West
Lansing-Westgate
Bedford Park-Nortown
St.Andrew-Windfields
Bridle Path-Sunnybrook-York Mills
Banbury-Don Mills
Victoria Village
Flemingdon Park
Parkwoods-Donalda
Pleasant View
Don Valley Village
Hillcrest Village
Bayview Woods-Steeles
Newtonbrook East
Willowdale East


In [16]:
#b = [i if "Restaurant" in i else 0 for i in toronto_venues["Venue Category"] ]
#b
toronto_venues["check"] = [i if "Restaurant" in i else float("nan") for i in toronto_venues["Venue Category"] ]
toronto_venues.dropna(axis=0,inplace = True)
toronto_venues.head()

,Neighbourhoods,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,check
2,West Humber-Clairville,43.716180,-79.596356,Mandarin Buffet,43.720360,-79.594387,Chinese Restaurant,Chinese Restaurant
3,West Humber-Clairville,43.716180,-79.596356,Xawaash,43.715786,-79.593053,Mediterranean Restaurant,Mediterranean Restaurant
7,West Humber-Clairville,43.716180,-79.596356,Swiss Pick,43.716150,-79.593843,Swiss Restaurant,Swiss Restaurant
15,Mount Olive-Silverstone-Jamestown,43.746868,-79.587259,NORI SUSHI,43.742775,-79.586985,Japanese Restaurant,Japanese Restaurant
16,Thistletown-Beaumond Heights,43.737988,-79.563491,Drupati's Doubles & Roti Shop,43.736222,-79.563471,Caribbean Restaurant,Caribbean Restaurant


In [18]:
toronto_venues.groupby('Neighbourhoods').count()
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
#toronto_onehot.drop(['Neighbourhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighbourhoods', value=toronto_venues['Neighbourhoods'] )
toronto_onehot.head()

,Neighbourhoods,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,North Indian Restaurant,Pakistani Restaurant,Persian Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,South American Restaurant,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
2,West Humber-Clairville,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,West Humber-Clairville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,West Humber-Clairville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
15,Mount Olive-Silverstone-Jamestown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,Thistletown-Beaumond Heights,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_grouped = toronto_onehot.groupby('Neighbourhoods').mean().reset_index()
toronto_grouped.shape

(85, 58)

In [31]:
toronto_grouped["total"] =toronto_grouped.loc[::,toronto_grouped.columns.to_list()[1::]].sum()

,Neighbourhoods,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,North Indian Restaurant,Pakistani Restaurant,Persian Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,South American Restaurant,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Agincourt North,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.500000,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.000000,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,Agincourt South-Malvern West,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.076923,0.000000,0.461538,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.076923,0.076923,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.076923,0.076923,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,Annex,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.2,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.2,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.2,0.000000
3,Banbury-Don Mills,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.500000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.500000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,Bay Street Corridor,0.0,0.043478,0.0,0.0,0.0,0.0,0.0,0.000000,0.043478,0.086957,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.043478,0.043478,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.130435,0.086957,0.0,0.0,0.000000,0.000000,0.043478,0.0,0.043478,0.0,0.0,0.0,0.0,0.043478,0.043478,0.086957,0.043478,0.0,0.086957,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.043478


In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [97]:
num_top_venues = 1

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhoods'] = toronto_grouped['Neighbourhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhoods,1st Most Common Venue
0,Agincourt North,Chinese Restaurant
1,Agincourt South-Malvern West,Chinese Restaurant
2,Annex,Vegetarian / Vegan Restaurant
3,Banbury-Don Mills,Italian Restaurant
4,Bay Street Corridor,Italian Restaurant


In [36]:
# set number of clusters
#dfToronto = dfToronto.drop("code_id",axis =1).rename(columns = {"Neighbourhoods" :"Neighbourhood"})
#toronto_grouped = pd.merge(dfToronto, toronto_grouped, on = "Neighbourhood")
kclusters = 5

toronto_clustering = df_new.drop(['Neighbourhoods',"code_id"],axis =1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 3, 3, 3, 3, 3, 3, 1, 0, 0, 1, 1, 3, 1, 2, 1, 1, 1, 1, 1, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 1, 1, 0, 0, 4, 1, 3, 3,
       3, 3, 3, 3, 1, 1, 3, 1, 3, 3, 3, 0, 1, 1, 1, 1, 3, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1,
       1, 1, 3, 3, 3, 1, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1, 0, 1, 3, 3, 3, 3,
       3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       1, 3, 3, 3, 3, 3, 3, 1], dtype=int32)

In [37]:
# add clustering labels
df_new["Cluster_Labels"] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merge = pd.merge(df_new,toronto_grouped,on = "Neighbourhoods")

In [59]:
# We choose place with population > 20000
toronto_population = toronto_merge[toronto_merge["Population_2016"] > 20000]

In [60]:
#Choose place with no 
toronto_open  = toronto_population[toronto_merge["Income_2016"] > 50000]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [101]:
toronto_open1 = toronto_open[["Neighbourhoods","Cluster_Labels","Population_2016","Income_2016","Longitude","Latitude","Vietnamese Restaurant"]]
toronto_open1

,Neighbourhoods,Cluster_Labels,Population_2016,Income_2016,Longitude,Latitude,Vietnamese Restaurant
7,Islington-City Centre West,1,43965,52787,-79.543317,43.633463,0.142857
38,Church-Yonge Corridor,1,31340,53583,-79.379017,43.659649,0.040000
28,East End-Danforth,1,21381,54324,-79.299359,43.684174,0.000000
39,Bay Street Corridor,1,25797,56526,-79.385721,43.657511,0.043478
61,Mount Pleasant West,1,29658,57039,-79.393360,43.704435,0.000000
36,Moss Park,1,20506,58915,-79.367297,43.656518,0.000000
16,Banbury-Don Mills,1,27695,67757,-79.349718,43.737657,0.000000
44,Niagara,1,31180,70623,-79.412420,43.636681,0.000000
29,The Beaches,0,21567,92580,-79.299601,43.671050,0.000000
54,Annex,0,30526,112766,-79.404001,43.671585,0.000000


### 4.0 Result
After all we have 10 best place base on business problem.

Place cluster 0 is high income than so this is good choice, however Islington-City Centre West is highest population is also good choice.

In [102]:
toronto_open_final = pd.merge(toronto_open1,neighbourhoods_venues_sorted,on = "Neighbourhoods")
toronto_open_final

,Neighbourhoods,Cluster_Labels,Population_2016,Income_2016,Longitude,Latitude,Vietnamese Restaurant,1st Most Common Venue
0,Islington-City Centre West,1,43965,52787,-79.543317,43.633463,0.142857,Fast Food Restaurant
1,Church-Yonge Corridor,1,31340,53583,-79.379017,43.659649,0.040000,Japanese Restaurant
2,East End-Danforth,1,21381,54324,-79.299359,43.684174,0.000000,Mexican Restaurant
3,Bay Street Corridor,1,25797,56526,-79.385721,43.657511,0.043478,Italian Restaurant
4,Mount Pleasant West,1,29658,57039,-79.393360,43.704435,0.000000,Italian Restaurant
5,Moss Park,1,20506,58915,-79.367297,43.656518,0.000000,Ethiopian Restaurant
6,Banbury-Don Mills,1,27695,67757,-79.349718,43.737657,0.000000,Italian Restaurant
7,Niagara,1,31180,70623,-79.412420,43.636681,0.000000,Thai Restaurant
8,The Beaches,0,21567,92580,-79.299601,43.671050,0.000000,Japanese Restaurant
9,Annex,0,30526,112766,-79.404001,43.671585,0.000000,Vegetarian / Vegan Restaurant


In [103]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_open_final['Latitude'], toronto_open_final['Longitude'], toronto_open_final['Neighbourhoods'], toronto_open_final['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [105]:
toronto_open_final.loc[toronto_open_final['Cluster_Labels'] == 0, toronto_open_final.columns[[1] + list(range(0, toronto_open_final.shape[1]))]]

,Cluster_Labels,Neighbourhoods,Cluster_Labels,Population_2016,Income_2016,Longitude,Latitude,Vietnamese Restaurant,1st Most Common Venue
8,0,The Beaches,0,21567,92580,-79.299601,43.671050,0.0,Japanese Restaurant
9,0,Annex,0,30526,112766,-79.404001,43.671585,0.0,Vegetarian / Vegan Restaurant
10,0,Bedford Park-Nortown,0,23236,123077,-79.420227,43.731486,0.0,Italian Restaurant


In [109]:
toronto_open_final.loc[toronto_open_final['Cluster_Labels'] == 1, toronto_open_final.columns[[1] + list(range(0, toronto_open_final.shape[1]))]]

,Cluster_Labels,Neighbourhoods,Cluster_Labels,Population_2016,Income_2016,Longitude,Latitude,Vietnamese Restaurant,1st Most Common Venue
0,1,Islington-City Centre West,1,43965,52787,-79.543317,43.633463,0.142857,Fast Food Restaurant
1,1,Church-Yonge Corridor,1,31340,53583,-79.379017,43.659649,0.040000,Japanese Restaurant
2,1,East End-Danforth,1,21381,54324,-79.299359,43.684174,0.000000,Mexican Restaurant
3,1,Bay Street Corridor,1,25797,56526,-79.385721,43.657511,0.043478,Italian Restaurant
4,1,Mount Pleasant West,1,29658,57039,-79.393360,43.704435,0.000000,Italian Restaurant
5,1,Moss Park,1,20506,58915,-79.367297,43.656518,0.000000,Ethiopian Restaurant
6,1,Banbury-Don Mills,1,27695,67757,-79.349718,43.737657,0.000000,Italian Restaurant
7,1,Niagara,1,31180,70623,-79.412420,43.636681,0.000000,Thai Restaurant


### 5.0 DISCUSSION
In general, I am positively impressed with the overall organization, content and lab works presented during the Coursera IBM Certification Course
I feel this Capstone project presented me a great opportunity to practice and apply the Data Science tools and methodologies learned.
I have created a good project that I can present as an example to show my potential.
I feel I have acquired a good starting point to become a professional Data Scientist and I will continue exploring to creating examples of practical cases.
### 6.0 CONCLUSIONS
I feel rewarded with the efforts, time and money spent. I believe this course with all the topics covered is well worthy of appreciation.
This project has shown me a practical application to resolve a real situation that has impacting personal and financial impact using Data Science tools.
The mapping with Folium is a very powerful technique to consolidate information and make the analysis and decision thoroughly and with confidence. I would recommend for use in similar situations.
One must keep abreast of new tools for DS that continue to appear for application in several business fields.